In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
from dotenv import load_dotenv

load_dotenv()


False

In [2]:
visits = pd.read_csv('./visits_1k.csv')
registrations = pd.read_csv('./regs_1k.csv')


In [5]:
visits['date'] = pd.to_datetime(visits['date'])
registrations['date'] = pd.to_datetime(registrations['date'])


In [ ]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

load_dotenv()

API_URL = os.getenv('API_URL')
DATE_BEGIN = os.getenv('DATE_BEGIN')
DATE_END = os.getenv('DATE_END')

response_visits = requests.get(
    f"{API_URL}/visits",
    params={'begin': DATE_BEGIN, 'end': DATE_END}
)
visits_data = response_visits.json()
visits_df = pd.DataFrame(visits_data)

response_regs = requests.get(
    f"{API_URL}/registrations",
    params={'begin': DATE_BEGIN, 'end': DATE_END}
)
regs_data = response_regs.json()
registrations_df = pd.DataFrame(regs_data)

visits_df['datetime'] = pd.to_datetime(visits_df['datetime'])
registrations_df['datetime'] = pd.to_datetime(registrations_df['datetime'])
